In [ ]:
import time
import datetime
import random
import pandas as pd
from tqdm import tqdm 

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd()  # 找出根目錄：Path.cwd()找出現在所在目錄(/run).parent(上一層是notebook).parent(再上層一層business_district_discovery)
print(project_root)
sys.path.append(str(project_root))

from newhouse591_spider import Newhouse591Spider
from config import regionid

In [ ]:
plvr_df = pd.read_pickle('../data/plvr_output_20250417.pkl')
presale_df = pd.read_pickle('../data/presale_output_20250417.pkl')

In [ ]:
plvr_df

In [ ]:
presale_df

In [ ]:
proc_df = pd.merge(
    plvr_df,
    presale_df[['縣市', '行政區', '社區名稱', '戶數', '銷售起始時間', '編號', '使用分區', '起造人', '建照執照']],
    on=['縣市', '行政區', '社區名稱'],
    how='left'
)

In [ ]:
proc_df

In [ ]:

unique_communities = proc_df[proc_df['建照執照'].isna()].drop_duplicates(
    subset=['縣市', '行政區', '社區名稱']
)[['縣市', '社區名稱']].reset_index(drop=True)


unique_communities['縣市'] = unique_communities['縣市'].replace(regionid)
# unique_communities

In [ ]:
search_list = unique_communities.to_dict('records')
len(search_list)

In [ ]:
newhouse591_spider = Newhouse591Spider()

In [ ]:
all_results = []
all_details = []
# 優化: 使用 tqdm 顯示進度條
for i, community in enumerate(tqdm(search_list[20:100], desc="搜尋進度")):
    try:
        # 優化: 直接取出字典中的值，而不是使用索引作為city_id
        filter_params = {
            'keyword': community['社區名稱'],  # 取出社區名稱
            'regionid': community['縣市'],     # 取出縣市代碼
        }
        
        # 搜尋建案
        print(f"正在搜尋: 第 {i} 個:{community['社區名稱']} (縣市代碼: {community['縣市']})")
        total_count, houses = newhouse591_spider.search(filter_params, {}, want_page=1)
        print(f"搜尋結果建案總數: {total_count}")
        
        # 優化: 確認是否有搜尋結果
        if houses and len(houses) > 0:
            # 取得建案詳情
            try:
                house_detail, main_df = newhouse591_spider.get_newhouse_detail(houses[0]['hid'])
                
                # 添加搜尋信息到DataFrame
                main_df['搜尋關鍵字'] = community['社區名稱']
                main_df['縣市代碼'] = community['縣市']
                
                # 儲存結果
                all_results.append(houses[0])
                all_details.append(main_df)
                
                print(f"已獲取 {houses[0].get('build_name', '未知')} 的詳細資料")
            except Exception as e:
                print(f"獲取詳細資料時發生錯誤: {e}")
        else:
            print(f"未找到與 {community['社區名稱']} 相關的建案")
    
    except Exception as e:
        print(f"搜尋時發生錯誤: {e}")
    
    # 添加延遲避免被封IP
    delay_time = random.uniform(3, 8)
    print(f"等待 {delay_time:.1f} 秒...")
    time.sleep(delay_time)

In [ ]:
combined_df = pd.concat(all_details, ignore_index=True)
combined_df

In [ ]:
today = datetime.datetime.now().strftime('%Y%m%d')
combined_df.to_csv(f'../data/output_591/combinde_df_20_100.csv', index=False, encoding='utf-8-sig')
combined_df.to_pickle(f'../data/output_591/combinde_df_20_100.pkl')

In [ ]:

filter_params = {
    # 關鍵字
    'keyword': '世界明珠',  # %E6%B0%B4%E6%BC%BE
    
    # 按地區
    'regionid': '1',  # (縣市) 
    # 'sectionid': '10',  # (地區) 台北 > 內湖區
    # 'shop_id': '164',  # (生活圈) 台北 > 內湖區 > 內湖科技園區
    
    # 按捷運 (有 regionid=1 參數，但應該沒影響)
    # 'search_type': '1',
    # 'city': '1',  # (捷運) 台北捷運
    # 'subway_line': '2',  # (路線) 台北捷運 > 淡水信義線
    # 'subway_id': '4198',  # (站點) 台北捷運 > 淡水信義線 > 新北投
    
    # 'unit_price': '1,2',  # (單價) 50萬/坪以下+50-70萬/坪
    # 'total_price': '3',  # (總價) 2000-3000萬
    # 'room': '4',  # (格局) 四房
    # 'shape': '3',  # (型態) 住宅大樓
    # 'build_type': '1',  # (狀態) 預售屋
    # 'purpose': '1',  # (用途) 住家用
    # 'tag': '1,4,5,10',  # (特色) 近捷運+重劃區+近公園+制震宅
}
sort_param = {
    # 'sort': '1',  # 金額排序從小到大
    # 'sort': '5',  # 瀏覽人數從多到少
}

In [ ]:
total_count, houses = newhouse591_spider.search(filter_params, sort_param, want_page=1)
print('搜尋結果建案總數：', total_count)
# with open('./newhouse591_search.json', 'w', encoding='utf-8') as f:
#     f.write(json.dumps(houses, ensure_ascii=False, indent=4))
#     print("新建案搜尋結果 已儲存至 ./newhouse591_search.json")

# time.sleep(random.uniform(2, 5))  # 隨機 delay 一段時間
house_detail, main_df = newhouse591_spider.get_newhouse_detail(houses[0]['hid'])
# house_detail = newhouse591_spider.get_newhouse_detail('135743')  # 120137
print(house_detail)
# with open('./newhouse591_detail.json', 'w', encoding='utf-8') as f:
#     f.write(json.dumps(house_detail, ensure_ascii=False, indent=4))
#     print("建案詳情 已儲存至 ./newhouse591_detail.json")

In [ ]:
main_df

In [ ]:
unique_communities = proc_df[proc_df['備查編號']==""]['社區名稱'].value_counts()
sorted_communities = unique_communities.sort_values(ascending=False)
print("缺失銷售起始時間的社區統計（從高到低）：")
for name, count in sorted_communities.items():
    print(f"{name}: {count}筆")

In [ ]:
# python# 創建一個包含更新資料的DataFrame
# update_data = pd.DataFrame([
#     {'縣市': '臺北市', '行政區': '中正區', '社區名稱': '中正城閱', '銷售起始時間': '1120315'},
#     # 可以加入更多需要更新的記錄...
# ])

# # 使用merge進行更新
# # 先把原始DataFrame和更新DataFrame合併
# temp_df = pd.merge(
#     merged_df,
#     update_data,
#     on=['縣市', '行政區', '社區名稱'],
#     how='left',
#     suffixes=('', '_update')
# )

# # 然後用更新值替換原始值（如果存在的話）
# mask = ~temp_df['銷售起始時間_update'].isna()
# temp_df.loc[mask, '銷售起始時間'] = temp_df.loc[mask, '銷售起始時間_update']

# # 刪除多餘的列並更新原始DataFrame
# merged_df = temp_df.drop(columns=['銷售起始時間_update'])